<a href="https://colab.research.google.com/github/kikiru328/CAKD3-Github/blob/main/Purchase_count_per_visit_pcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)


Mounted at /content/drive


In [2]:
#  Directory 변경 (모듈이용)
%pwd
%cd '../content/notebooks'

# module import
## Data Load
from pickle5 import pickle 

## Basic module
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

import warnings
warnings.filterwarnings('ignore')

/content/drive/My Drive/Colab Notebooks


In [3]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/data/EDA/기존_purprd_cust.pkl','rb') as file:
  df = pickle.load(file)


In [4]:
df_1 = df[(df['구매년도']==2014) & (df['구매월']>=1) & (df['구매월']<=6)]
df_2 = df[(df['구매년도']==2014)&(df['구매월']>=7) & (df['구매월']<=12)]
df_3 = df[(df['구매년도']==2015)&(df['구매월']>=1) & (df['구매월']<=6)]
df_4 = df[(df['구매년도']==2015)&(df['구매월']>=7) & (df['구매월']<=12)]

In [28]:
def countpur(df,a):
  countpur = df[['고객번호','영수증번호','구매일자','구매금액']]
  countpur = countpur.groupby(['고객번호','구매일자','영수증번호']).sum()
  countpur = countpur.reset_index()
  countpur['거래횟수'] = 1 
  countpur = countpur.groupby('고객번호').sum()
  countpur = countpur.drop(['영수증번호','구매금액'], axis=1)
  countpur = countpur.rename(columns={'거래횟수':'방문별거래횟수'})
  cols = list(countpur.columns)
  for i in cols:
    countpur[f'{i}_{a}'] = countpur[i]
  countpur = countpur.drop(cols,axis=1)
  return countpur


In [33]:
H1 = countpur(df_1,1)
H2 = countpur(df_2,2)
H3 = countpur(df_3,3)
H4 = countpur(df_4,4)

In [34]:
H1

방문별거래횟수_1
고객번호            
1            159
2            188
3            128
4            137
5            220
...          ...
19356         12
19357          7
19359          5
19360         20
19364         36

[19085 rows x 1 columns]

In [37]:
def train_df(a,b):
  train = pd.concat([a,b],axis=1)
  return train

train = train_df(H1,H2)
train = train.reset_index()

def test_df(a,b,c):
    A = pd.concat([a,b],axis=1)
    test = pd.concat([A,c],axis=1)
    return test

test = test_df(H1,H2,H3)  
test = test.reset_index()

In [38]:
display(train.head())
display(test.head())

고객번호  방문별거래횟수_1  방문별거래횟수_2
0     1        159        219
1     2        188        167
2     3        128        168
3     4        137        105
4     5        220         75

고객번호  방문별거래횟수_1  방문별거래횟수_2  방문별거래횟수_3
0     1        159        219        185
1     2        188        167        166
2     3        128        168         94
3     4        137        105        134
4     5        220         75         81

In [39]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/train/PCV.pkl', 'wb') as train_save:
  pickle.dump(train,train_save)
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/test/PCV.pkl', 'wb') as test_save:
  pickle.dump(test,test_save)

In [40]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/train/PCV.pkl', 'rb') as train_save:
  tr = pickle.load(train_save)
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/1012발표도못하고수정/10월14일이후 새로추가/test/PCV.pkl', 'rb') as test_save:
  tt = pickle.load(test_save)

In [41]:
tr

고객번호  방문별거래횟수_1  방문별거래횟수_2
0          1        159        219
1          2        188        167
2          3        128        168
3          4        137        105
4          5        220         75
...      ...        ...        ...
19080  19356         12        115
19081  19357          7        178
19082  19359          5        207
19083  19360         20        125
19084  19364         36        142

[19085 rows x 3 columns]

In [42]:
tt

고객번호  방문별거래횟수_1  방문별거래횟수_2  방문별거래횟수_3
0          1        159        219        185
1          2        188        167        166
2          3        128        168         94
3          4        137        105        134
4          5        220         75         81
...      ...        ...        ...        ...
19080  19356         12        115        171
19081  19357          7        178        202
19082  19359          5        207        151
19083  19360         20        125         88
19084  19364         36        142        135

[19085 rows x 4 columns]